<a href="https://colab.research.google.com/github/jlrocam/my-first-binder/blob/main/lesiones_piel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Preparación del entorno
#Crear las carpetas
import os
data = "/content/data"
#train_images = "/content/data"
models =  "/content/models"
os.makedirs(data, exist_ok=True)
#os.makedirs(train_images, exist_ok=True)
os.makedirs(models, exist_ok=True)

In [ ]:
#Montar el Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Subir el fichero zip con las imágenes y descomprimir
!cp "/content/drive/MyDrive/Colab Notebooks/datos/train_images.zip" /content/data
!unzip /content/data/train_images.zip -d /content/data
!rm /content/data/train_images.zip
#Comprobar que las imagenes están en /data/train_images

Se han truncado las últimas 5000 líneas del flujo de salida.
  inflating: /content/data/train_images/ISIC_0068028.jpg  
  inflating: /content/data/train_images/ISIC_0068029.jpg  
  inflating: /content/data/train_images/ISIC_0068030.jpg  
  inflating: /content/data/train_images/ISIC_0068031.jpg  
  inflating: /content/data/train_images/ISIC_0068032.jpg  
  inflating: /content/data/train_images/ISIC_0068033.jpg  
  inflating: /content/data/train_images/ISIC_0068034.jpg  
  inflating: /content/data/train_images/ISIC_0068035.jpg  
  inflating: /content/data/train_images/ISIC_0068036.jpg  
  inflating: /content/data/train_images/ISIC_0068037.jpg  
  inflating: /content/data/train_images/ISIC_0068038.jpg  
  inflating: /content/data/train_images/ISIC_0068039.jpg  
  inflating: /content/data/train_images/ISIC_0068040.jpg  
  inflating: /content/data/train_images/ISIC_0068041.jpg  
  inflating: /content/data/train_images/ISIC_0068042.jpg  
  inflating: /content/data/train_images/ISIC_0068043.j

In [ ]:
##Comprobar que el archivo metadata.csv está en /content/data
#El archivo está en train_images y se debe mover a /content/data
!mv /content/data/train_images/metadata.csv /content/data

#Crear el fichero del modelo skin_lesion_model.keras en /models
!touch /content/models/skin_lesion_model.keras

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Configuración de TensorFlow
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

# Directorios de datos
DATASET_DIR = "/content/data/train_images"
METADATA_FILE = "/content/data/metadata.csv"
MODEL_PATH = "/content/models/skin_lesion_model.keras"

# Verificar la ruta de las imágenes
image_files = [f for f in os.listdir(DATASET_DIR) if os.path.isfile(os.path.join(DATASET_DIR, f))]
print(f"Encontradas {len(image_files)} imágenes en {DATASET_DIR}")
print("Primeros 10 nombres de archivo de imagen:", image_files[:10])

# Cargar metadatos
df = pd.read_csv(METADATA_FILE)

# Preprocesar los datos
df['isic_id'] = df['isic_id'].apply(lambda x: x + '.jpg')

# Codificar las etiquetas
le_1 = LabelEncoder()
le_3 = LabelEncoder()
df['diagnosis_1_encoded'] = le_1.fit_transform(df['diagnosis_1'])
df['diagnosis_3_encoded'] = le_3.fit_transform(df['diagnosis_3'])

# Función para generar datos
def generate_data(df, directory):
    for _, row in df.iterrows():
        img_path = os.path.join(directory, row['isic_id'])
        img = cv2.imread(img_path)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # Normalización
        yield img, (row['diagnosis_1_encoded'], row['diagnosis_3_encoded'])

# Dividir el DataFrame en entrenamiento y validación
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Crear datasets
output_signature = (
    tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
    (tf.TensorSpec(shape=(), dtype=tf.int32),
     tf.TensorSpec(shape=(), dtype=tf.int32))
)

train_dataset = tf.data.Dataset.from_generator(
    lambda: generate_data(train_df, DATASET_DIR),
    output_signature=output_signature
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: generate_data(val_df, DATASET_DIR),
    output_signature=output_signature
)

# Preparar los datasets
BATCH_SIZE = 16
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Crear el modelo
inputs = Input(shape=(224, 224, 3))
x = Conv2D(32, (3,3), activation="relu", kernel_regularizer=l2(0.01))(inputs)
x = MaxPooling2D(2,2)(x)
x = Conv2D(64, (3,3), activation="relu")(x)
x = MaxPooling2D(2,2)(x)
x = Conv2D(128, (3,3), activation="relu")(x)
x = MaxPooling2D(2,2)(x)
x = Flatten()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.5)(x)
output1 = Dense(len(le_1.classes_), activation="softmax", name="diagnosis_1")(x)
output2 = Dense(len(le_3.classes_), activation="softmax", name="diagnosis_3")(x)

model = Model(inputs=inputs, outputs=[output1, output2])

# Compilar el modelo
model.compile(optimizer="adam",
              loss={"diagnosis_1": "sparse_categorical_crossentropy",
                    "diagnosis_3": "sparse_categorical_crossentropy"},
               metrics={"diagnosis_1": "accuracy", "diagnosis_3": "accuracy"})

# Entrenamiento
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10
)

# Guardar modelo
model.save(MODEL_PATH)
print("Modelo guardado en", MODEL_PATH)

model.save("/content/drive/MyDrive/skin_lesion_model.keras")


Encontradas 18946 imágenes en /content/data/train_images
Primeros 10 nombres de archivo de imagen: ['ISIC_0067959.jpg', 'ISIC_0053491.jpg', 'ISIC_0054633.jpg', 'ISIC_0059286.jpg', 'ISIC_0055378.jpg', 'ISIC_0064129.jpg', 'ISIC_0058265.jpg', 'ISIC_0055382.jpg', 'ISIC_0059066.jpg', 'ISIC_0070966.jpg']
Epoch 1/10
    948/Unknown 110s 108ms/step - diagnosis_1_accuracy: 0.5063 - diagnosis_1_loss: 1.0888 - diagnosis_3_accuracy: 0.3146 - diagnosis_3_loss: 1.9766 - loss: 3.1040

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


948/948 ━━━━━━━━━━━━━━━━━━━━ 136s 135ms/step - diagnosis_1_accuracy: 0.5064 - diagnosis_1_loss: 1.0887 - diagnosis_3_accuracy: 0.3146 - diagnosis_3_loss: 1.9765 - loss: 3.1039 - val_diagnosis_1_accuracy: 0.5984 - val_diagnosis_1_loss: 0.9510 - val_diagnosis_3_accuracy: 0.4303 - val_diagnosis_3_loss: 1.7192 - val_loss: 2.6990
Epoch 2/10
948/948 ━━━━━━━━━━━━━━━━━━━━ 113s 119ms/step - diagnosis_1_accuracy: 0.5876 - diagnosis_1_loss: 0.9503 - diagnosis_3_accuracy: 0.4182 - diagnosis_3_loss: 1.7318 - loss: 2.7100 - val_diagnosis_1_accuracy: 0.6129 - val_diagnosis_1_loss: 0.9310 - val_diagnosis_3_accuracy: 0.4501 - val_diagnosis_3_loss: 1.6783 - val_loss: 2.6334
Epoch 3/10
948/948 ━━━━━━━━━━━━━━━━━━━━ 140s 117ms/step - diagnosis_1_accuracy: 0.6024 - diagnosis_1_loss: 0.9265 - diagnosis_3_accuracy: 0.4563 - diagnosis_3_loss: 1.6588 - loss: 2.6084 - val_diagnosis_1_accuracy: 0.6216 - val_diagnosis_1_loss: 0.9086 - val_diagnosis_3_accuracy: 0.4694 - val_diagnosis_3_loss: 1.6281 - val_loss: 2.55

Una vez entrenado el modelo, veamos sus resultados

In [3]:
import tensorflow as tf
import os

# Ruta del modelo
MODEL_PATH = "/content/models/skin_lesion_model.keras"

# Verificar si el archivo existe
if os.path.exists(MODEL_PATH):
    try:
        # Cargar el modelo
        model = tf.keras.models.load_model(MODEL_PATH)
        print("✅ El modelo se ha cargado correctamente.")

        # Mostrar estructura del modelo
        model.summary()
        print(model.output_shape)

    except Exception as e:
        print("❌ Error al cargar el modelo:", e)
else:
    print(f"❌ El archivo {MODEL_PATH} no existe.")


✅ El modelo se ha cargado correctamente.


Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_20            │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_60 (Conv2D)        │ (None, 222, 222, 32)   │            896 │ input_layer_20[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_60          │ (None, 111, 111, 32)   │              0 │ conv2d_60[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_61 (Conv2D)        │ (None, 109, 109, 64)   │         18,496 │ max_pooling2d_60[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_61          │ (None, 54, 54, 64)     │              0 │ conv2d_61[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_62 (Conv2D)        │ (None, 52, 52, 128)    │         73,856 │ max_pooling2d_61[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_62          │ (None, 26, 26, 128)    │              0 │ conv2d_62[0][0]        │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_2 (Flatten)       │ (None, 86528)          │              0 │ max_pooling2d_62[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_20 (Dense)          │ (None, 128)            │     11,075,712 │ flatten_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_20 (Dropout)      │ (None, 128)            │              0 │ dense_20[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ diagnosis_1 (Dense)       │ (None, 4)              │            516 │ dropout_20[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ diagnosis_3 (Dense)       │ (None, 11)             │          1,419 │ dropout_20[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,512,687 (127.84 MB)

 Trainable params: 11,170,895 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,341,792 (85.23 MB)

[(None, 4), (None, 11)]


In [13]:
#Prueba del modelo
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Ruta del modelo
MODEL_PATH = "/content/models/skin_lesion_model.keras"

# Cargar el modelo
model = tf.keras.models.load_model(MODEL_PATH)

# Cargar los LabelEncoders (asegúrate de tener estos disponibles o recrearlos)
# Recrear los LabelEncoders
METADATA_FILE = "/content/data/metadata.csv"
# Cargar metadatos
data_info = pd.read_csv(METADATA_FILE)

# Cargar metadatos
le_1 = LabelEncoder()
le_3 = LabelEncoder()

# Ajustar los LabelEncoders a los datos originales
le_1.fit(data_info['diagnosis_1'])
le_3.fit(data_info['diagnosis_3'])

# Función para preprocesar la imagen
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalización
    return np.expand_dims(img, axis=0)  # Añadir dimensión batch

# Función para realizar predicciones
def predict_image(image_path):
    # Preprocesar la imagen
    img = preprocess_image(image_path)

    # Realizar la predicción
    predictions = model.predict(img)  # Esto devuelve una lista con dos arrays (uno por salida)

    # Obtener las probabilidades para cada diagnóstico
    diagnosis_1_prob = predictions[0]  # Predicciones para diagnosis_1
    diagnosis_3_prob = predictions[1]  # Predicciones para diagnosis_3

    # Obtener las clases predichas
    diagnosis_1_class = le_1.inverse_transform([np.argmax(diagnosis_1_prob)])[0]
    diagnosis_3_class = le_3.inverse_transform([np.argmax(diagnosis_3_prob)])[0]

    return {
        "diagnosis_1": {
            "class": diagnosis_1_class,
            "probability": float(np.max(diagnosis_1_prob))
        },
        "diagnosis_3": {
            "class": diagnosis_3_class,
            "probability": float(np.max(diagnosis_3_prob))
        }
    }

# Ejemplo de uso
image_path = '/content/data/ISIC_0053453.jpg'  # Reemplaza con la ruta de tu imagen
result = predict_image(image_path)
print("Predicción:")
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
Predicción:
{'diagnosis_1': {'class': 'Malignant', 'probability': 0.4085986912250519}, 'diagnosis_3': {'class': 'Basal cell carcinoma', 'probability': 0.2532508075237274}}


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Precisión del entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión de la validación')
plt.title('Evolución de la precisión')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(loc='lower right')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# Obtener las predicciones del modelo en el conjunto de validación
y_pred = model.predict(validation_generator)
y_pred = (y_pred > 0.5).astype(int)  # Convertir las probabilidades a etiquetas binarias (0 o 1)

# Obtener las etiquetas reales del conjunto de validación
y_true = validation_generator.classes

#Calculamos matriz de confusión
cm = confusion_matrix(y_true, y_pred)

NameError: name 'validation_generator' is not defined

In [ ]:
#Visualizacion de matriz de confusión
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.xlabel("Predicciones")
plt.ylabel("Etiquetas reales")
plt.title("Matriz de confusión")
plt.show()

In [ ]:
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

print(f"Precisión del entrenamiento: {train_accuracy[-1]:.4f}")
print(f"Precisión de la validación: {val_accuracy[-1]:.4f}")

KeyError: 'accuracy'

In [ ]:
# --- Liberar memoria ---
del df, train_datagen, train_generator, validation_generator
import gc
gc.collect()

## **A partir de aquí se introduce código para entrenamiento incremental o fine-tuning**

In [ ]:
import os
data = "/content/data"
models =  "/content/models"
os.makedirs(data, exist_ok=True)
os.makedirs(models, exist_ok=True)

In [8]:
#Subir el fichero zip con las imágenes y descomprimir y renombrar carpeta a incremental
!unzip /content/data/incremental.zip -d /content/data
#!rm /content/data/incremental.zip
#Comprobar que las imagenes están en /data/incremental y el metadata también

Se han truncado las últimas 5000 líneas del flujo de salida.
 extracting: /content/data/ISIC_0025250.jpg  
 extracting: /content/data/ISIC_0029627.jpg  
 extracting: /content/data/ISIC_0032872.jpg  
 extracting: /content/data/ISIC_0032286.jpg  
 extracting: /content/data/ISIC_0035938.jpg  
 extracting: /content/data/ISIC_0034562.jpg  
 extracting: /content/data/ISIC_0034352.jpg  
 extracting: /content/data/ISIC_0026548.jpg  
 extracting: /content/data/ISIC_0031392.jpg  
 extracting: /content/data/ISIC_0031046.jpg  
 extracting: /content/data/ISIC_0029754.jpg  
 extracting: /content/data/ISIC_0031910.jpg  
 extracting: /content/data/ISIC_0024362.jpg  
 extracting: /content/data/ISIC_0031958.jpg  
 extracting: /content/data/ISIC_0027702.jpg  
 extracting: /content/data/ISIC_0034784.jpg  
 extracting: /content/data/ISIC_0025876.jpg  
 extracting: /content/data/ISIC_0028590.jpg  
 extracting: /content/data/ISIC_0025245.jpg  
 extracting: /content/data/ISIC_0029315.jpg  
 extracting: /conte

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

# Rutas de datos
DATASET_DIR = "/content/data"  # 📌 Carpeta con nuevas imágenes
METADATA_FILE = "/content/data/metadata.csv"  # 📌 Nuevo CSV con etiquetas
MODEL_PATH = "/content/models/skin_lesion_model.keras" # 📌 Modelo previamente entrenado


# Cargar el modelo preentrenado
model = load_model(MODEL_PATH)
print("✅ Modelo cargado correctamente.")

# Cargar metadatos nuevos
df = pd.read_csv(METADATA_FILE)
# Preprocesar los datos
df['isic_id'] = df['isic_id'].apply(lambda x: x + '.jpg')

print(f"Número de imágenes a procesar: {len(df)}")

# Preprocesar etiquetas
le_1 = LabelEncoder()
le_3 = LabelEncoder()
df["diagnosis_1_encoded"] = le_1.fit_transform(df["diagnosis_1"])
df["diagnosis_3_encoded"] = le_3.fit_transform(df["diagnosis_3"])


# Generador de datos usando tf.data.Dataset
def generate_data(df, directory):
    for _, row in df.iterrows():
        img_path = os.path.join(directory, row['isic_id'])
        img = cv2.imread(img_path)
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # Normalización
        yield img, (row['diagnosis_1_encoded'], row['diagnosis_3_encoded'])


# Crear dataset de TensorFlow
output_signature = (
    tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
    (tf.TensorSpec(shape=(), dtype=tf.int32),
     tf.TensorSpec(shape=(), dtype=tf.int32))
)

new_dataset = tf.data.Dataset.from_generator(
    lambda: generate_data(df, DATASET_DIR),
    output_signature=output_signature
)

# Preparar el dataset
BATCH_SIZE = 16
new_dataset = new_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Reentrenar el modelo con las nuevas imágenes
history = model.fit(
    new_dataset,
    epochs=5  # 🔄 Número de épocas para refinamiento
)

# Guardar el modelo actualizado
model.save(MODEL_PATH)
print("✅ Modelo actualizado y guardado en", MODEL_PATH)

model.summary()


✅ Modelo cargado correctamente.
Número de imágenes a procesar: 11720
Epoch 1/5
    261/Unknown 3135s 12s/step - diagnosis_1_accuracy: 0.7825 - diagnosis_1_loss: 0.6120 - diagnosis_3_accuracy: 0.5007 - diagnosis_3_loss: 1.6694 - loss: 2.2942

In [7]:
!rm -r /content/data/incremental/